# Chocolate Milkshake Betaseries 


## Table of Conents:

* [Load and set variables](#first-bullet)
* [FSL feat1 analysis](#second-bullet)  
* [Concatenate with fslmerge](#third-bullet)
* [Transform functionals with flirt and fslmath](#fourth-bullet)
* [Pull individual ROI timeseries by subject](#fifth-bullet)


**The Paradigm**

Functional Tasks: 

    milkshake pic, milkshake receipt, h20 pic, h20 receipt, milkshake minus h20, milkshake plus h20, rinse

Furthermore, `milkshake` is broken down into the following specifities:   
      
    HF(high fat, HS(high sugar), and respectively, LF(low fat), LS(low sugar)   

with the following relationships:  
    
    HF_HS, HF_LS, LF_HS, LF_LS


Example set of onsets for a subject:  


`
$ /pwd ~/preprocessed/sub-001/ses-1/func/onsets
mkB_h20_pic.ev          mkB_HS_plus_h20.ev      mkB_rinse.ev            mkC_HF_LS_receipt.ev    mkC_milkshake_pic.ev
mkB_h20_receipt.ev      mkB_LF_HS_minus_h20.ev  mkC_h20_pic.ev          mkC_HS_plus_h20.ev      mkC_rinse.ev
mkB_HF_HS_minus_h20.ev  mkB_LF_HS_receipt.ev    mkC_h20_receipt.ev      mkC_LF_HS_minus_h20.ev
mkB_HF_HS_receipt.ev    mkB_LF_LS_receipt.ev    mkC_HF_HS_minus_h20.ev  mkC_LF_HS_receipt.ev
mkB_HF_LS_receipt.ev    mkB_milkshake_pic.ev    mkC_HF_HS_receipt.ev    mkC_LF_LS_receipt.ev`

In [3]:
import glob
import os
import pandas as pd
import argparse
import subprocess
import re
from IPython.core import display as ICD
from os import listdir
from shutil import rmtree
from subprocess import check_output

## Load and set variables <a class="anchor" id="first-bullet"></a>

In [2]:
# load and set variables
beta_path='/projects/niblab/experiments/chocolate_milkshake/data/betaseries'
concat_path="/projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat"
subject_folders=sorted(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/bids/derivatives/preprocessed/sub-*'))

data_dict={}
bad_subs=[]

for folder in subject_folders:
    subject_id=folder.split("/")[-1]
    #print(os.path.join(folder,'ses-1/analysis/beta/*.feat'))
    tasks=glob.glob(os.path.join(folder,'ses-1/analysis/beta/*'))
    #print(tasks)
    if not tasks:
        bad_subs.append(subject_id)
    else:
        if subject_id not in data_dict:
            data_dict[subject_id]={}
        for task_folder in tasks:
            #print(task_folder)
            task=task_folder.split("/")[-1].split('.')[0]
            #print(task)
            if task not in data_dict[subject_id]:
                data_dict[subject_id][task]={}
            pes=glob.glob(os.path.join(task_folder, '*.feat'))
            data_dict[subject_id][task]=pes
                

## FSL feat1 analysis <a class='anchor' id='second-bullet'></a>

In [29]:
choco_evs=glob.glob('/projects/niblab/data/eric_data/ev_files/milkshake/*.ev')
choco_evs=[x.split('/')[-1].replace('.ev',"") for x in choco_evs]
choco_evs=[re.sub('mk[A-Z]_', '',x) for x in choco_evs]
choco_evs=set(choco_evs)
print("[INFO] set of unique chocolate evs: \n", choco_evs)

[INFO] set of unique chocolate evs: 
 {'HF_HS_minus_h20', 'HF_HS_receipt', 'LF_HS_minus_h20', 'h20_pic', 'HS_plus_h20', 'rinse', 'milkshake_pic', 'h20_receipt', 'HF_LS_receipt', 'LF_LS_receipt', 'LF_HS_receipt'}


In [15]:
# get the unique tasks
!ls /projects/niblab/data/eric_data/ev_files

GNG  GNG_wrong	imagine  milkshake


### Make subject condition feat files (`.fsf`)

In [ ]:
def make_file(sub_id, ses_id, trial_id, output_dir, task,data_dict):
    with open(os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/code/beta_design.fsf'),'r') as infile:
        tempfsf=infile.read()
        #
        if not os.path.exists(os.path.join(output_dir, "design_files")):
            os.makedirs(os.path.join(output_dir, "design_files"))
        #print(output_dir)
        design_fileout = os.path.join(output_dir, "design_files/%s_%s_%s_feat1.fsf"%(sub_id, ses_id, trial_id))
        out_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["OUTPUT"]
        func_param = data_dict[sub_id][task]["FUNCRUN"]
        confound_param = data_dict[sub_id][task]["CONFOUND"]
        trial_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["TRIAL"]
        nuis_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["NUIS"]

        tempfsf = tempfsf.replace("OUTPUT", out_param)
        tempfsf = tempfsf.replace("FUNCRUN", func_param) 
        tempfsf = tempfsf.replace("CONFOUND", confound_param)
        tempfsf = tempfsf.replace("TRIAL", trial_param)
        tempfsf = tempfsf.replace("NUIS", nuis_param)

        for i in range(6):
            moco = data_dict[sub_id][task]["MOCO%i"%i]
            tempfsf = tempfsf.replace("MOCO%i"%i, moco)
        try:
            with open(design_fileout,'w') as outfile:
                outfile.write(tempfsf)
            outfile.close()
        except:
            print("BAD SUBJECT ", sub_id)
        infile.close()
        
        

## Concatenate (fslmerge) data <a class='anchor' id='third-bullet'></a>

### Grab Concatenated Data

In [3]:
# grab concatenated (fslmerge) data

fslmerged_files=glob.glob(os.path.join(concat_path,'*.nii.gz'))
fslmerged_files[:4]

['/projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat/sub-001_task-milkshakeB.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat/sub-001_task-milkshakeC.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat/sub-004_task-milkshakeA.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat/sub-004_task-milkshakeB.nii.gz']

In [4]:
# fill dictionary with concatenated files
for nifti in fslmerged_files:
    task=nifti.split("/")[-1].split("_")[1].split('.')[0]
    subject_id=nifti.split("/")[-1].split("_")[0]
    key=nifti.split("/")[-1].split(".")[0].split('_')[1]
    key=key+"_concat"
    #print(subject_id)
    #data_dict[subject_id][task]['concat']=nifti

## Transform Functionals (flirt & fslmaths) <a class='anchor' id='fourth-bullet'></a> 
To use the BigBrain300 rois we need to transform the nifti task files to match the mask.
ROIs location on renci: `/projects/niblab/parcellations/chocolate_decoding_rois`   

Big Brain 300: `/projects/niblab/parcellations/chocolate_decoding_rois/old_rois/bigBrain300_atlas`
  
`flirt`: the main program that performs affine registration. The options we use here:  
* `-in`, an input  
* `-ref`, a reference volume  
* `applyxfm`, `-init` and `-out`, apply a saved transformation to a volume   

  
  
*For these usages the reference volume must still be specified as this sets the voxel and image dimensions of the resulting volume.*



In [5]:
print('[INFO] transform afunctionals to match the mask.')

reference_nifti='/projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.nii.gz'
reference_mat='/projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.mat'
for nii in fslmerged_files:
    
    # setup and run flirt
    nii=nii.replace('.nii.gz', '')
    out=nii+'_3mm'
    flirt_cmd="flirt -in {} -ref {} -init {} -applyxfm -out {}".format(nii, reference_nifti, reference_mat, out)
    #print('[INFO] flirt command: \n{}'.format(flirt_cmd))
    #os.system(flirt_cmd)
    
    fslmaths_cmd='fslmaths {} -thr 0.9 {}'.format(out,out)
    #print('[INFO] fslmaths command: \n{}'.format(fslmaths_cmd))
    #os.system(fslmaths_cmd)
    
print('[INFO] transformation process complete.')

    
    

[INFO] transform afunctionals to match the mask.
[INFO] transformation process complete.


## Pull ROI timeseries  <a class='anchor' id='fifth-bullet'></a>

Pull individual rois timeseries for each subject. 

Example command:  


    fslmeants -i ~/sub-001_punish.nii.gz -o ~/3_pull_timeseries/sub-001_punish_AI_35_23_-6_asymPREPspace.nii.gz.txt -m ~/AI_35_23_-6_asymPREPspace.nii.gz



Here is the code:

In [6]:

def chunks(l,n):
    return [l[i:i+n] for i in range(0, len(l), n)]


def pull_timeseries(roi_list, bb300_path='/projects/niblab/parcellations/bigbrain300',roi_df='/projects/niblab/parcellations/bigbrain300/renaming.csv'):

    
    bad_subs=[]
    #ICD.display(roi_df)

    # load asymmetrical nifti roi files
    asym_niftis=glob.glob("/projects/niblab/parcellations/bigbrain300/MNI152Asymmetrical_3mm/*.nii.gz")

    # load roi list
    out_dir = os.path.join(beta_path, 'rois/big300')
    #print('[INFO] OUT DIRECTORY: %s \n'%out_dir)

    #roi_df.set_index("final order name", inplace=True)
    #ICD.display(roi_df)#.head())

    # run parallel job pools


    # loop through the roi file list
    #print(roi_list[:3])
    for nifti in sorted(roi_list):
        #print('[INFO] loop1')
        subj_id = nifti.split("/")[-1].split("_")[0]
        subj_condition=nifti.split("/")[-1].split("_")[1].split(".")[0]
        #print('[INFO] roi: %s %s'%(subj_id, subj_condition))

        # loop through roi reference list
        for ref_nifti in sorted(asym_niftis):
            #print('[INFO] reference roi: %s'%ref_nifti)
            roi = ref_nifti.split('/')[-1].split(".")[0]
            out_path = os.path.join(out_dir, "{}_{}_{}.txt".format(subj_id, subj_condition, roi))
            #print(roi, out_path)
            cmd='fslmeants -i {} -o {} -m {}'.format(nifti, out_path, ref_nifti)
            try:
                cmd='fslmeants -i {} -o {} -m {}'.format(nifti, out_path, ref_nifti)
                #print("Running shell command: {}".format(cmd))
                #os.system(cmd)
            except:
                bad_subs.append((subj_id, subj_condition))
        
        #print('[INFO] finished processing for %s'%subj_id)


    return "%s"%bad_subs

   

In [7]:
from multiprocessing import Pool
# load roi
print("[INFO] loading roi and reference file....")

# task rois loaded
task_rois=glob.glob(os.path.join(concat_path,'*_3mm.nii.gz'))
#task_rois[:5]
print("[INFO] {} task roi nifti files being processed.".format(len(task_rois)))

chunksize=10
print("[INFO] chunksize: {}".format(chunksize))
chunk_list=chunks(task_rois, chunksize)
#roi_df['network']
# pull timeseries by rois --fslmeants command


with Pool(5) as p:
    p.map(pull_timeseries, chunk_list)
print("[INFO] process complete.")



[INFO] loading roi and reference file....
[INFO] 224 task roi nifti files being processed.
[INFO] chunksize: 10
[INFO] process complete.


**Submit process as a batch job for large datasets**  

Two files:  
- timeseries_pull.job  
- timeseries_pull.py  

*quick view of file contents below* -- note,for now paths may need to be updated directly in the scripts for your unqiue setup, flexible script in progress.

 

Batch file:

In [8]:
!cat /projects/niblab/experiments/chocolate_milkshake/data/code/timeseries_pull.job

#!/bin/bash
#SBATCH --job-name=timeseriespull
#SBATCH -o /projects/niblab/experiments/chocolate_milkshake/data/error_files/timepull_out_%A.txt
#SBATCH -e /projects/niblab/experiments/chocolate_milkshake/data/error_files/timepull_beta_err_%A.txt



/projects/niblab/modules/software/miniconda3/miniconda3/bin/python3 /projects/niblab/experiments/chocolate_milkshake/data/code/timeseries_pull.py


Python file:

In [9]:
!cat /projects/niblab/experiments/chocolate_milkshake/data/code/timeseries_pull.py

import glob
import os
import pandas as pd
import argparse
import subprocess
import re

from IPython.core import display as ICD
from os import listdir
from shutil import rmtree
from subprocess import check_output


subject_folders=sorted(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/bids/derivatives/preprocessed/sub-*'))
beta_path='/projects/niblab/experiments/chocolate_milkshake/data/betaseries'
concat_path="/projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat"

fslmerged_files=glob.glob(os.path.join(concat_path,'*.nii.gz'))
fslmerged_files[:4]


data_dict={}
bad_subs=[]


for folder in subject_folders:
    subject_id=folder.split("/")[-1]
    #print(os.path.join(folder,'ses-1/analysis/beta/*.feat'))
    tasks=glob.glob(os.path.join(folder,'ses-1/analysis/beta/*'))
    #print(tasks)
    if not tasks:
        bad_subs.append(subject_id)
    else:
        if subject_id not in data_dict:
            data_dict[subject_id]={}
        for task_fol

To submit job file:  

In [10]:
#!sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/timeseries_pull.job

To view job:

In [11]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


### Get timeseries roi task files 

In [12]:
# view timeseries 
listdir('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/big300')[:5]


['sub-049_task-milkshakeC_bb300_MNI152Asymm3mm_001.txt',
 'sub-001_task-milkshakeB_bb300_MNI152Asymm3mm_001.txt',
 'sub-112_task-milkshakeD_bb300_MNI152Asymm3mm_001.txt',
 'sub-097_task-milkshakeC_bb300_MNI152Asymm3mm_001.txt',
 'sub-020_task-milkshakeC_bb300_MNI152Asymm3mm_001.txt']

In [13]:
# view number of subject timeseries rois created/pulled
len(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/big300/*.txt'))

67200

## Combine Timeseries into Matrix  <a class="anchor" id="sixth-bullet"></a>
Combine timeseries roi files into, **one file per condition per participant**  

In [14]:
subject_ids=list(data_dict.keys())
subject_ids[-3:]


['sub-150', 'sub-151', 'sub-154']

### Create Matrix

Loop through subject ids and combine in matrix

In [10]:
def create_subject_matrices(subjects):
    for subject_id in subjects:
        tasks=list(data_dict[subject_id].keys())
        for task in tasks:

            # get roi texts for subject / condition
            roi_txts = glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/big300/%s_%s*txt'%(subject_id,task))
            df_lst=[]
            for txt in roi_txts: 
                #print(txt)
                df_temp = pd.read_csv(txt, sep="\n", header=None)
                #print(df_temp)
                df_lst.append(df_temp)
            try:
                df_concat= pd.concat(df_lst, axis=1, sort=False)
                #print(df_concat)

                # write output file 
                outfile='/projects/niblab/experiments/chocolate_milkshake/data/betaseries/subject_matrices/%s_%s-receipt.txt'%(subject_id,task)
                #print('[INFO] making file %s'%outfile)
                #df_concat.to_csv(outfile, header=None, index=None, sep='\t')
            except:
                print('[INFO]error with %s condition %s, passing...'%(subject_id,task))
        



In [29]:
roi_txts = glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/big300/%s_%s*txt'%(subject_id,task))
outfile='/projects/niblab/experiments/chocolate_milkshake/data/betaseries/subject_matrices/%s_%s.txt'%(subject_id,task)

subject_chunks=chunks(subject_ids, 10)
#subject_chunks[:2]

print("[INFO] processing %s lists"%len(subject_chunks))


[INFO] processing 12 lists


Run parallel process:

In [31]:
print("[INFO] making subject condition matrix from rois timeseries...")

with Pool(10) as p:
    p.map(create_subject_matrices, subject_chunks)
    
print("[INFO] completed process.")



[INFO] making subject condition matrix from rois timeseries...
[INFO]error with sub-088 condition task-milkshakeD, passing...
[INFO]error with sub-032 condition task-milkshakeD, passing...
[INFO]error with sub-033 condition task-milkshakeC, passing...
[INFO]error with sub-015 condition task-milkshakeC, passing...
[INFO]error with sub-081 condition task-milkshakeC, passing...
[INFO] completed process.


Quick view of new files:

In [34]:
listdir('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/subject_matrices')[-5:]

['sub-142_task-milkshakeB-receipt.txt',
 'sub-143_task-milkshakeC-receipt.txt',
 'sub-143_task-milkshakeA-receipt.txt',
 'sub-144_task-milkshakeD-receipt.txt',
 'sub-144_task-milkshakeA-receipt.txt']

In [35]:
len(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/subject_matrices/*.txt'))

224